# 05: Delaunay Triangulation and Voronoi Diagrams

*Authors: Lars Nitzschke, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Data Structures    
    3.1. Slab Decomposition  
    3.2. Vertical Decomposition
4. References  

## 1. Introduction

TODO: Introduction (Def., as statet in the lecture; explanation of the examples below)

TODO: Images

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [ ]:
# Python standard library imports
from abc import ABC, abstractmethod
from typing import Any, Optional, Iterable
import random
import numpy as np

# Data structure, geometry and visualisation module imports
from modules.data_structures import Comparator, ComparisonResult as CR, Vertex, HalfEdge, DoublyConnectedEdgeList, PointLocation, Face, VDLineSegment, AnimationBinaryTreeDict, PLSearchStructure
from modules.geometry import Point, PointReference, PointSequence, LineSegment, EPSILON, Rectangle, VerticalOrientation as VORT, HorizontalOrientation as HORT, Orientation as ORT
from modules.visualisation import VisualisationTool, PointLocationInstance, VerticalExtensionMode, PointLocationMode, PointSetInstance, DCELInstance, DCELMode

Additionally, we create an object for our visualisation tool and register a few example instances.

In [ ]:
visualisation_tool = VisualisationTool(400, 400, PointSetInstance())
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

c = 0.5 * canvas_size
r = 0.9 * c
s = c - r
t = c + r
u = (t - s) / 36

points = [Point(s + 20*u, t -  1*u), Point(s + 25*u, t -  6*u), Point(s + 31*u, t -  9*u), Point(s + 28*u, t - 11*u),
          Point(s + 19*u, t - 14*u), Point(s + 30*u, t - 21*u), Point(s + 27*u, t - 29*u), Point(s + 22*u, t - 33*u),
          Point(s + 13*u, t - 31*u), Point(s +  8*u, t - 34*u), Point(s +  2*u, t - 30*u), Point(s + 11*u, t - 24*u),
          Point(s +  5*u, t - 22*u), Point(s +  1*u, t - 17*u), Point(s +  4*u, t - 14*u), Point(s +  6*u, t -  6*u),
          Point(s + 10*u, t -  3*u), Point(s + 15*u, t - 17*u), Point(s + 18*u, t - 27*u)]

visualisation_tool.register_example_instance("Example Point Set", points)

## 3. Algorithms

### 3.1 Delaunay Triangulation

In [ ]:
visualisation_tool.display()

In [ ]:
# Construction of the Delaunay Triangulation
class DelaunayTriangulation(DoublyConnectedEdgeList):
    def __init__(self, points: Iterable[Point] = []):
        self.clear()
        # Initialize triangulation with big triangle consisting of points p_-1 = (3M, 0), p_-2 = (0, 3M), p_-3 = (-3M, -3M)
        # where M is the max abs value of any coordinate of any point
        self.max_coord = 0
        for point in points:
            if abs(point.x) > self.max_coord:
                self.max_coord = abs(point.x)
            if abs(point.y) > self.max_coord:
                self.max_coord = abs(point.y)        
        surrounding_triangle_points = [Point(3*self.max_coord, 0), Point(0, 3*self.max_coord), Point(-3*self.max_coord, -3*self.max_coord)]
        self.surrounding_triangle = [self.add_vertex(point) for point in surrounding_triangle_points]
        self._add_edge(self.surrounding_triangle[0], self.surrounding_triangle[1])
        self._add_edge(self.surrounding_triangle[0], self.surrounding_triangle[2])
        self._add_edge(self.surrounding_triangle[1], self.surrounding_triangle[2])
        
        # Add all other points
        for point in points:
            self.insert(point)

    def insert(self, point: Point) -> None:
        containing_triangle = self.find_containing_face(point)  # TODO: Replace by search structure
        # search returns triangle if point is inside and edge if point is on it.
        on_edge: HalfEdge = None
        inside: bool = True
        if inside:
            vertex = self.add_vertex(point)  # TODO: Maybe with less checks that the point is correct?
            # Case inside triangle:
            surrounding_vertices = list(containing_triangle.outer_vertices())
            if len(surrounding_vertices) != 3:
                raise Exception("Face is not a triangle")
            edge_1 = self._add_edge(vertex, surrounding_vertices[0], return_edge = True)
            edge_2 = self._add_edge(vertex, surrounding_vertices[1], return_edge = True)
            edge_3 = self._add_edge(vertex, surrounding_vertices[2], return_edge = True)
            self.legalize_edge(vertex, edge_1.next)
            self.legalize_edge(vertex, edge_2.next)
            self.legalize_edge(vertex, edge_3.next)
            # TODO: Update Search Structure: split
        else:
            vertex = self.add_vertex_in_edge(on_edge, point)  # TODO: Maybe with less checks
            edge_1 = self._add_edge(vertex, vertex.edge.next.destination, return_edge = True)
            edge_2 = self._add_edge(vertex, on_edge.twin.next.destination, return_edge = True)
            self.legalize_edge(vertex, edge_1.twin.prev)
            self.legalize_edge(vertex, edge_1.next)
            self.legalize_edge(vertex, edge_2.twin.prev)
            self.legalize_edge(vertex, edge_2.next)
            # TODO: Update Search Structure: split

    def legalize_edge(self, vertex: Vertex, edge: HalfEdge) -> None:
        opposing_vertex: Vertex = edge.twin.next.destination
        if self.is_edge_illegal(vertex, edge, opposing_vertex):  # Edge flip
            self.remove_edge(edge)
            self._add_edge(vertex, opposing_vertex)  # Add flipped edge

            self.legalize_edge(vertex, edge.twin.next)
            self.legalize_edge(vertex, edge.twin.prev)
            # TODO: Update Search Structure: flip

    def is_edge_illegal(self, vertex: Vertex, edge: HalfEdge, opposing_vertex: Vertex, epsilon: float = EPSILON) -> bool:
        # If the two opposing triangles do not form a convex quadrilateral, the edge can not be illegal. See Lemma 9.4
        if not self._convex_quadrilateral(vertex, edge, opposing_vertex, epsilon):
            return False      
        # Handling special cases with the surrounding triangle of points p_0, p_-1, p_-2 TODO: Maybe check with coord comparison to 3M
        surrounding_vertices = [v in self.surrounding_triangle for v in [edge.destination, vertex, edge.origin, opposing_vertex]]
        if surrounding_vertices[0] and surrounding_vertices[2]:  # Case (i): We must keep the edges of the surrounding triangle
            return False
        count = sum(surrounding_vertices)
        if count == 0:  # Case (ii): Normal case
            # Lemma 9.4 [1] / Lemma 1 [lecture 05] circumcircle C around triangle <=> illegal if inside C
            # Assume the points are counter clockwise
            return self._in_circumcircle(vertex, edge, opposing_vertex, epsilon)
        if count == 1:  # Case (iii): No special point should destroy actual delaunay edges: Replace edge if one of the special points
            return surrounding_vertices[0] or surrounding_vertices[2]
        if count == 2:  # Case (iv): One vertex of the edge and one of the other vertices
            edge_index =  -1 - self.surrounding_triangle.index(edge.destination if surrounding_vertices[0] else edge.origin)
            other_index = -1 - self.surrounding_triangle.index(vertex if surrounding_vertices[1] else opposing_vertex)
            return edge_index > other_index
        else:  # Case (v): count == 3 is impossible 
            raise ValueError(f"Illegal edge test with wrong parameters.")
        
    def _convex_quadrilateral(self, vertex: Vertex, edge: HalfEdge, opposing_vertex: Vertex, epsilon: float = EPSILON) -> bool:
        return opposing_vertex.point.orientation(vertex.point, edge.origin.point,      epsilon) == ORT.LEFT \
           and opposing_vertex.point.orientation(vertex.point, edge.destination.point, epsilon) == ORT.RIGHT
    
    def _in_circumcircle(self, vertex: Vertex, edge: HalfEdge, opposing_vertex: Vertex, epsilon: float = EPSILON) -> bool:
        det = np.linalg.det(np.array([[edge.destination.x, edge.destination.y, edge.destination.x*edge.destination.x + edge.destination.y*edge.destination.y, 1],
                                      [vertex.x,           vertex.y,           vertex.x*vertex.x + vertex.y*vertex.y,                                         1],
                                      [edge.origin.x,      edge.origin.y,      edge.origin.x*edge.origin.x + edge.origin.y*edge.origin.y,                     1],
                                      [opposing_vertex.x,  opposing_vertex.y,  opposing_vertex.x*opposing_vertex.x + opposing_vertex.y*opposing_vertex.y,    1]]))
        if abs(det) < epsilon:
            det = 0
        return det > 0

random_seed = 42
def delaunay_triangulation(points: set[Point]) -> PointSequence:
    points: list[Point] = list(points)
    if random_seed is not None:
        random.seed(random_seed)
    random.shuffle(points)  # Randomized Incremental Construction
    triangulation: DelaunayTriangulation = DelaunayTriangulation(points)
    raw_points_triangulation = DCELInstance.extract_points_from_raw_instance(triangulation)
    ps = PointSequence()
    for point in raw_points_triangulation:
        ps.append(point)
    return ps

In [ ]:
visualisation_tool.register_algorithm("Delaunay", delaunay_triangulation, DCELMode())
visualisation_tool.display()